In [ ]:
'''
y is the sequence of tags while x is the observed sequence of words 
Code dies when doing CN and SG files.
'''

In [4]:
'''
General functions
read & write requires encoding='utf8' due to different languages in the files.
'''
from math import log 
#read the training data file 
def read_file_train(fp):
    print("reading train file...")
    parsed = []
    f = open(fp,'r',encoding='utf8')
    while True:
        line = f.readline() 
        #clean up the data read 
        if line == '\n':
            parsed.append(line.replace('\n','Blank Blank')) #Replace spaces with Blanks.
        elif line == "":
            break
        else:
            parsed.append(line.strip()) #remove \n at the end 
        
    return parsed

#read the test data file 
def read_file_test(fp):
    print("reading tests file...")
    parsed = []
    f = open(fp,'r',encoding='utf8')
    while True:
        line = f.readline() 
        #clean up the data read 
        if line == '\n':
            parsed.append(line.replace('\n','Blank')) #Replace spaces with Blanks.
        elif line == "":
            break
        else:
            parsed.append(line.strip()) #remove \n at the end 
        
    return parsed

'''
data received shall be in dictionary term of 
data = ((x1 ,y),(x2,y),...)
fn = filename 
fp = filepath
'''
def write_file(fp,fn,data):
    print("writing file...")
    with open(fp+fn, 'w',encoding='utf8') as f:
        for k in data:
            if k[0] == "Blank": #replace the blanks
                f.write("\n")
            else:
                f.write(k[0] + ' ' + k[1]+'\n')
    f.close()
    

# build dictionaries of count(y->x) and count(y)
def built_dict(train_data):
    print("building count(y->x) & count(y)...")
    '''
    data will be a array of the read train data 
    1. create tuples:count out of the y and x and put them in global_y_x
    2. if the tuples were repeated, don't add a new entry into the dictionary but increase the count 
 
    1. from global_y_x, we have gotten count(y-> x) . 
    2. need get count(y)
    
    returns huge_count_y_x and huge_count_y 
    huge_count_y_x = {("We","B-Positive"):1}
    huge_count_y = {"B-Positive":1}
    '''
    
    huge_count_y_x = {} #dict to contain all count(y->x)
    huge_count_y = {} #dict to contain all count(y)
    for d in train_data:
        x,y = d.split(" ") #split by spaces 
        temp_tuple = (x,y)
        
        if temp_tuple not in huge_count_y_x:
            huge_count_y_x[temp_tuple] = 1
        else:
            huge_count_y_x[temp_tuple] = huge_count_y_x[temp_tuple] + 1 
        
        if y not in huge_count_y:
            huge_count_y[y] =  1
        else: 
            huge_count_y[y] = huge_count_y[y] + 1
    return huge_count_y_x,huge_count_y 

#part 2 first part 

def calculate_emmision(huge_count_y_x, huge_count_y):
    print("calculating emission probability...")
    '''
    Calculating the emission 
    Use the tuple (x,y) to represent the occurance of e(x|y)  
    '''
    huge_emission ={} #dict to contain all the emission probabilities 
    for seq, count_y_x in huge_count_y_x.items():
        
        emission = 0.0 #if none found, then return as 0 
        tag = seq[1]
#         print(k,v)
        count_y = huge_count_y[tag] 
        emission = float(count_y_x / count_y)
#         emission = float(log(count_y_x) - log(count_y +1))
        huge_emission[seq] = emission

    return huge_emission

# parsed_train = read_file_train("EN/train")
# print(calculate_emmision(parsed_train))

#part 2 second part 
'''
    replace words that appear < k times with #UNK# 
    modify_train_set accepts k, training data
    training data is a list returned from read_file_train
'''
def modify_train_set(k,train_data):
    print("modifying to #UNK#...")
    modified_training_data = []
    count_dict = {}
    for td in train_data:
#         print(td)
        word,tag = td.rsplit(" ",1) 
        if word not in count_dict:
            count_dict[word] = 1
        else:
            count_dict[word] = count_dict[word] + 1
    #identify which words appeared less than k times 
    list_of_works_less_than_k = [] 
    for w,count in count_dict.items():
        if count < k and w != "Blank": #Skip blanks. 
            list_of_works_less_than_k.append(w)
    #now replace the entries in the training set  
    for td in train_data:
        word,tag = td.rsplit(" ",1)
        for words_to_be_removed in list_of_works_less_than_k:
            if word == words_to_be_removed:
                index = train_data.index(td)
                train_data[index] = '#UNK# ' + tag
    return train_data
                
#part 2 third part 
'''
using the emission data, predict the possible tags (?) 
the tuple (x,y) is key to occurance of e(x|y) 
TODO:
1. replace with #UNK# 
2. for each word w in the test_data
    3. find the tuples in huge_emissison that matches w
    4. if there's more than one, select the one with max(emission probability)
    5. if there's only 1 result, then use that tag 
    6. append the tag to y_star
7. write the results of w, tag into a tuple
8. return the array y_star of tuples

y_star = [("We","O"),("are","I"),...]
'''
def simple_sentiment_analysis(test_data,modified_train_data,huge_emission):
    print("doing simple_sentiment_analysis...")
    #extract the words 
    extracted_train_words = []
    for d in modified_train_data:
        x,y = d.rsplit(" ",1)
        extracted_train_words.append(x)
    #check whether the word exists in train. else replace with #UNK# 
    for w in test_data:
        if w not in extracted_train_words and w != "Blank": #Avoid blanks.
            index = test_data.index(w)
            test_data[index] = '#UNK#'
    y_star=[] #the tags, to be appended here 
    for w in test_data:
        temp_dict = {} # to hold the possible results found from huge_emission 
        for key,e_value in huge_emission.items():
            
            if w == key[0]: 
                temp_dict[key] = e_value
        #now return the tag that has the max e probability

        best_result = max(temp_dict, key=temp_dict.get)[1]

        #append a tupple for word, tag

        y_star.append((w,best_result))
    #return the dictionary -- don't do dic cos will remove duplicates 

    return y_star

#part 3 first part 
'''
first modify the train_data to get START and STOP in the data.
train data = ['We A','are B','young C','Blank Blank','Hello D'] from read_file_train(fp+"train")
test data = ['We','are','young','Blank','Hello'] from read_file_test(fp+"dev.in")
Change to 
train data = ['START','We A','are B','young C','STOP','START','Hello D','STOP'] 
test data = ['START','We','are','young','STOP','START','Hello','STOP']
'''
def mod_data_for_transition(td,data_type):
    print("Inserting START and STOP for " + data_type +"...")
    wtd = [] #words from train_data
    if data_type == "train":
        #extract the words first 
        for d in td:
            w,t = d.rsplit(" ",1) 
            wtd.append(t)
    else:
        wtd = td #don't need to extract 
    
    #start replacing 
    for i in range(len(wtd)):
        if wtd[i] == 'Blank':
            index = i
            wtd[index] = "STOP"
            wtd.insert(index+1,"START")
    wtd.insert(0,"START")
    wtd[len(wtd)-1]="STOP"
    #TODO: am not sure why we need to do it twice. 
    for w in wtd:
        if w == "Blank":
            index = wtd.index(w)
            wtd[index] = "STOP"
            wtd.insert(index+1,"START")
    return wtd 
    
    
    
def build_dict_transition(td): 
    print("constructing dictionaries for transition...")
    '''
    td=["START","O","O","STOP","START",...]
    1. build a tuple for (yi,yi-1)
    2. put in dict 
    huge_count_yi_1_yi = {(yi,yi-1):1,...}
    3. extract key[1] and put in huge_count_yi_1 
    huge_count_yi_1 = {"yi-1":1,...}
    '''
    huge_count_yi_1_yi = {} #stores the no. of times of transition 
    huge_count_yi_1 = {} #stores no of times yi-1 occured 
#     print(len(td))
    for i in range(1,len(td)):
        # i = 0,1,2,...,n 
        temp = (td[i-1],td[i])
        if temp not in huge_count_yi_1_yi:
            huge_count_yi_1_yi[temp] = 1
        else:
            huge_count_yi_1_yi[temp] = huge_count_yi_1_yi[temp] + 1 
#         print (temp)
        if td[i-1] not in huge_count_yi_1:
            huge_count_yi_1[td[i-1]] = 1
        else:
            huge_count_yi_1[td[i-1]] = huge_count_yi_1[td[i-1]] + 1
            
    return huge_count_yi_1_yi,huge_count_yi_1

def calculate_transition(huge_count_yi_1_yi,huge_count_yi_1):
    print("calculating transition probabilities...")
    '''
    huge_transition = {(yi_1,yi):1,...}
    '''
    huge_transition = {}
    for yi_1_yi,count_yi_1_yi in huge_count_yi_1_yi.items():
        transition = 0.0 #ensure float
        for yi_1,count_yi_1 in huge_count_yi_1.items(): 
            if yi_1 == yi_1_yi[0]:
                transition = float(count_yi_1_yi / count_yi_1) #ensure float 
#                 transition = float(log(count_yi_1_yi) - log(count_yi_1))
                huge_transition[yi_1_yi] = transition
    return huge_transition

#nightmare part 3 second part: viterbi algo. 
#td is test data 
def viterbi(emission,transition,td):  
    print("Doing Viterbi...")
    massive_y_star = [] #contains tuples of word and tags 
    TAGS = ['O','B-positive','B-neutral','B-negative','I-positive','I-negative','I-neutral'] #ALL TAGS IN. 
    #split into individual sentences
    seperated_sentence = []
    temp = []
#     all_pi = {}
    for word in td:
        if word != 'STOP':
            temp.append(word)
        else:
            temp.append("STOP")
            seperated_sentence.append(temp)
            temp =[]
    #START OF VITERBI#
    #try one sentence:
#     fs = seperated_sentence[1]
#     for base case y0 -> y1 
    for fs in seperated_sentence:
#         print("\nDoing new sentence")
#         print(fs)
        y_star = [] #contains tags 
#         print("Doing START")
        temp_pi={}
        temp_all_tags_score = {}
        all_pi = {}
        for v in TAGS:
            max_pi_k_v =0

            pi_prev = 0.0 
            isBased = check_basecase(fs[0])
            if isBased :
                pi_prev = 1.0
            emission_value = 0.0
            transition_value = 0.0
            ekey = (fs[1],v)
            tkey = ('START',v)
            if ekey in emission:
                emission_value = emission[(fs[1],v)]
            if tkey in transition:
                transition_value = transition[tkey]
#             print("em: ",emission_value,"t: ",transition_value)
            one_pi = pi_prev * emission_value * transition_value
            temp_all_tags_score[v] = one_pi
    #     print(temp_all_tags_score)    
        all_pi[1] = temp_all_tags_score

#         print("Doing forward recursive")
        for k in range(2,len(fs)-1): #ignore STOP
#             print("\nyi-1=",fs[k-1],"yi=",fs[k])
            temp_all_tags_score = {}
            for v in TAGS:
                max_pi_k_v = 0.0
                temp_pi = {}

                for u in TAGS:
                    emission_current = 0.0 
                    transition_current = 0.0 
                    ekey = (fs[k],v)
                    tkey = (u,v)
    #                 print(ekey)
    #                 print(tkey)
                    if ekey in emission:
                        emission_current = emission[ekey]
                    if tkey in transition:
                        transition_current = transition[tkey]
                    #base case 
                    pi_prev = 0.0 
                    if k-1 == 0: 
                        isBased = check_basecase(fs[k-1])
                        if isBased:
                            pi_prev = 1.0 
                    else:
                        pi_prev = all_pi[k-1][u]
    #                 one_pi = pi_prev + emission_current + transition_current
                    one_pi = pi_prev * emission_current * transition_current
    #                 print(u,"->",v,"Pi[k-1,v]: ",pi_prev," emission: ",emission_current," transition: ", transition_current," pi[k,v]: ", one_pi)
                    temp_pi[u] = one_pi
    #             print("\nFINISHED COMPARING PREV NODES FOR CURRENT NODE:",v," ",temp_pi)
    #             print("\ncapturing max score to node ",v , "for ", fs[k]) 
                max_pi_k_v = max(temp_pi.values())
    #             print(max_pi_k_v)
                temp_all_tags_score[v] = max_pi_k_v
    #             print("allocated:", temp_all_tags_score)
    #             print("\nDoing next node.")
            all_pi[k] = temp_all_tags_score
    #         print("\n")
#         print("Doing STOP case")
        #for last case, yn -> yn+1 = STOP 
        k = len(fs) - 2
        temp_pi_stop={}
        temp_all_tags_score_stop = {}
    #     print("Doing ",fs[k],"->","STOP")
#         print(all_pi)
        for v in TAGS:
            max_pi_k_v =0 
            pi_prev = 0.0 
            emission_value = 0.0
            transition_value = 0.0
            tkey = (v,'STOP') #fs[k+1] = STOP
            if tkey in transition:
                transition_value = transition[tkey]
#             print(len(fs),len(fs) - 2,k,k-1)
            if len(fs) == 3:
                pi_prev = all_pi[k][v]
            else:
                pi_prev = all_pi[k-1][v]
            one_pi = pi_prev * transition_value
#             print("t p: ",transition_value,"pi[k-1,v]: ",pi_prev,"pi[n+1,v]: ",one_pi)
            temp_pi_stop[v] = one_pi 
    #         print(temp_pi_stop)
        max_pi_k_v = max(temp_pi_stop.values())  
        all_pi[len(fs)-1] = max_pi_k_v
        
#         print("Doing backwards STOP")
        #backwards 
        y_star.append('STOP')
        #for STOP 
        n = len(fs) - 2 
    #     pi_stop = all_pi[n]
        yn_star_values= {}
        for v in TAGS:
            transition_value = 0.0
            tkey = (v,'STOP') #fs[k+1] = STOP
            if tkey in transition:
                transition_value = transition[tkey]
            pi_current = all_pi[n][v] * transition_value
            yn_star_values[v] = pi_current
        yn_star = max(yn_star_values,key = yn_star_values.get)
        y_star.insert(0,yn_star)

#         print("Doing backwards")
        for n in range(len(fs)-3,1,-1): #don't include START 
            yn_1_star_values = {}
#             print("\ndoing word: ",fs[n])
            for u in TAGS:
                transition_value = 0.0
                tkey = (u,y_star[0]) 
#                 print(tkey[0],"->",tkey[1])
                if tkey in transition:
                    transition_value = transition[tkey]
    #             print("transS: ",transition_value)
#                 print("pi[n-1,v]: ", all_pi[n][u])
                pi_current = all_pi[n][u] * transition_value
                yn_1_star_values[u] = pi_current
#             print("all the values ",yn_1_star_values)
            yn_1_star = max(yn_1_star_values, key = yn_1_star_values.get)
    #         print(yn_1_star)
            y_star.insert(0,yn_1_star)

        #do START -> y1 
#         print("Doing backwards START")
        n = 1 
        yn_star_values_start= {}
        for v in TAGS:
#             print("\ndoing START->",v,"for word ",fs[n])
            transition_value = 0.0
            tkey = ('START',v)
            if tkey in transition:
                transition_value = transition[tkey]
            pi_current = all_pi[n][v] * transition_value
            yn_star_values_start[v] = pi_current
        yn_star_start = max(yn_star_values_start, key = yn_star_values_start.get)
        y_star.insert(0,yn_star_start)
#         print("\n")
#         print(all_pi)
#         print("\n")
        y_star.insert(0,"START")
#         print(fs)
#         print(y_star)
        massive_y_star.append(y_star)
#     print(massive_y_star)
    massive_y_star = process_viterbi_results(massive_y_star,seperated_sentence)
    return massive_y_star

def check_basecase(state):
    if state=='START':
        return True
    else:
        return False

def mod_testdata(test_data,modified_train_data):
    print("Replacing to #UNK# for test...")
    #extract the words 
    extracted_train_words = []
    for d in modified_train_data:
        x,y = d.rsplit(" ",1)
        extracted_train_words.append(x)
    #check whether the word exists in train. else replace with #UNK# 
    for w in test_data:
        if w not in extracted_train_words and w != "START" and w!= "STOP": #Avoid START and STOP.
            index = test_data.index(w)
            test_data[index] = '#UNK#'
    return test_data

#returns in (word,tag), with START STOP repleaced, Blank returns. 
def process_viterbi_results(massive_y_star,seperated_sentence):
    tagged_words = [] #return this 
    for i in range(len(massive_y_star)):
        for j in range(len(massive_y_star[i])):
            if massive_y_star[i][j] == "START":
                continue #don't do anything 
            elif massive_y_star[i][j] == "STOP":
                #replace to blank blank 
                temp = ('Blank','Blank')
                tagged_words.append(temp)
            else: #normal words 
                temp = (seperated_sentence[i][j],massive_y_star[i][j])
                tagged_words.append(temp)
    return tagged_words

    
    

## Part 4

In [60]:
TAGS = ['O','B-positive','B-neutral','B-negative','I-positive','I-negative','I-neutral'] #ALL TAGS IN. 

#applicable to all cases 
def alpha_base(transition):
    alpha_base = {}
    for u in TAGS:
        if ('START',u) in transition:
            alpha_base[u] = transition[('START',u)]
        else:
            alpha_base[u] = 0.0
    return alpha_base

#tag u from Y node_no , where word is from Xn from the list of words 
def beta_base(transition,emission,sentence):
    X_n = sentence[len(sentence)-2] #last word 
    beta_base = {} 
    for u in TAGS:
        transition_v = 0.0 
        emission_v = 0.0 
        if (u,'STOP') in transition:
            transition_v = transition[(u,'STOP')]
        if (X_n,u) in emission:
            emission_v = emission[(X_n,u)]
        beta_base[u] = float(transition_v * emission_v)
    return beta_base


def split_sentence(td):
    print("Splitting sentence...") 
    #split into individual sentences
    seperated_sentence = []
    temp = []
    for word in td:
#         print(word == "STOP")
        if word != "STOP":
            temp.append(word)
        else:
            temp.append("STOP")
#             print(temp)
            seperated_sentence.append(temp)
            temp =[]
    return seperated_sentence

def max_marginal(td,transition,emission,alpha_base):
    sentences = split_sentence(td)
    sentence = sentences[0] #test on one sentence first 
    print(sentence)
    betaBase = beta_base(transition,emission,sentence) #get the beta base 
    forward = {} 
    forward[1] = alpha_base #this should be constant
    YSTAR = [] 
    #now do forward 
    for j in range(1,len(sentence)-1):
#         print(sentence[j])
        temp_alphas = {}
        for u in TAGS:
            alpha_j_1_v = 0.0 #value of alpha u (j+1) 
            for v in TAGS:
                a = 0.0 
                b = 0.0 
                if (v,u) in transition:
                    a = transition[(v,u)]
                if (sentence[j],v) in emission:
                    b = emission[(sentence[j],v)]
                alpha_j_1_v += float(forward[j][v] * a * b )
            temp_alphas[u] = alpha_j_1_v
        forward[j+1]= temp_alphas
#     return forward
    #now do backward
    backwards = {}
    backwards[len(sentence)-1] = betaBase # inserting beta u n
    for j in range(len(sentence)-2,0,-1): #j = n-1 ,...,1 
        temp_beta={}
        for u in TAGS: 
            beta_u_j = 0.0 
            for v in TAGS:
                a = 0.0 
                b = 0.0 
                if (u,v) in transition:
                    a = transition[(u,v)]
                if (sentence[j],u) in emission:
                    b = emission[(sentence[j],u)]
                beta_u_j += float(backwards[j+1][v]*a*b)
            temp_beta[u] = beta_u_j
        backwards[j] = temp_beta

#     return backwards
    #find the tags
    for j in range(1,len(sentence)-1):

#         YSTAR.append(max(forward[j],key=forward.get) * max(backwards[j].values(),key=backwards.get))
        temp = {}
        for u in TAGS:
            temp[u] = forward[j][u] * backwards[j][u]
        YSTAR.append(max(temp,key=temp.get))
    return YSTAR

In [61]:
#code to run the file 
PART2_FN = "dev.p2.out"
PART3_FN = "dev.p3.out"
PART4_FN = "dev.p4.out"
EN_FP = "EN/"
FR_FP = "FR/"
CN_FP = "CN/"
SG_FP = "SG/"
# FPList = [EN_FP,FR_FP,CN_FP,SG_FP]
FPList = [EN_FP]

In [62]:
#part 4
for fp in FPList:
    print("\nDoing " + fp + "...")
    parsedtrainData = read_file_train(fp+"train")
    # print(parsedtrainData)
    modifiedData = modify_train_set(3,parsedtrainData)
    stsp_train = (mod_data_for_transition(modifiedData,"train"))
#     print(stsp_train)
    huge_count_y_x, huge_count_y = built_dict(modifiedData)
    huge_emission = calculate_emmision(huge_count_y_x, huge_count_y)
    huge_count_yi_1_yi,huge_count_yi_1 = build_dict_transition(stsp_train)
#     print(huge_count_yi_1)
    huge_transition = calculate_transition(huge_count_yi_1_yi,huge_count_yi_1)
#     print(huge_transition)
#     td = huge_transition
    parsedtestData = read_file_test(fp+"dev.in")
    stsp_test = mod_data_for_transition(parsedtestData,"test")
    modifiedTestdata = mod_testdata(stsp_test,modifiedData)
#     transition,emission,sentence
#     print(split_sentence(modifiedTestdata)[0])
#     print(beta_base(huge_transition,huge_emission,split_sentence(modifiedTestdata)[0]))
    print(max_marginal(modifiedTestdata,huge_transition,huge_emission,alpha_base(huge_transition)))
    print("End...\n")


Doing EN/...
reading train file...
modifying to #UNK#...
Inserting START and STOP for train...
building count(y->x) & count(y)...
calculating emission probability...
constructing dictionaries for transition...
calculating transition probabilities...
reading tests file...
Inserting START and STOP for test...
Replacing to #UNK# for test...
Splitting sentence...
['START', 'When', 'I', 'called', 'this', '#UNK#', ',', 'I', "didn't", 'think', 'I', 'would', 'be', 'able', 'to', 'get', 'in', 'at', '#UNK#', ',', 'but', 'I', 'was', 'able', 'to', 'get', 'in', ',', '#UNK#', 'with', 'four', 'other', 'guests', '.', 'STOP']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
End...



In [ ]:
#part 3 
print("Part 3 running...")
td ={}
for fp in FPList:
    print("\nDoing " + fp + "...")
    parsedtrainData = read_file_train(fp+"train")
    # print(parsedtrainData)
    modifiedData = modify_train_set(3,parsedtrainData)
    stsp_train = (mod_data_for_transition(modifiedData,"train"))
#     print(stsp_train)
    huge_count_y_x, huge_count_y = built_dict(modifiedData)
    huge_emission = calculate_emmision(huge_count_y_x, huge_count_y)
    
    huge_count_yi_1_yi,huge_count_yi_1 = build_dict_transition(stsp_train)
#     print(huge_count_yi_1)
    huge_transition = calculate_transition(huge_count_yi_1_yi,huge_count_yi_1)
#     print(huge_transition)
    td = huge_transition
    parsedtestData = read_file_test(fp+"dev.in")
    
    stsp_test = mod_data_for_transition(parsedtestData,"test")
    modifiedTestdata = mod_testdata(stsp_test,modifiedData)
#     print(modifiedTestdata)
    massive_y_star =viterbi(huge_emission,huge_transition,modifiedTestdata)
    print(massive_y_star)
#     write_file(fp,PART3_FN,massive_y_star)
    print("End...\n")

# for k,v in td.items():
#     print(k,v)



In [205]:
#part 2
print("Part 2 running...\n")
he = {}
for fp in FPList:
    print("\nDoing " + fp + "...")
    parsedtrainData = read_file_train(fp+"train")
    # print(parsedtrainData)
    modifiedData = modify_train_set(3,parsedtrainData)
    # print(modifiedData)
    huge_count_y_x, huge_count_y = built_dict(modifiedData)
#     print(huge_count_y)
    huge_emission = calculate_emmision(huge_count_y_x, huge_count_y)
    he = huge_emission
    parsedtestData = read_file_test(fp+"dev.in")
    # print(parsedtestData)
    y_star = simple_sentiment_analysis(parsedtestData,modifiedData,huge_emission)
    # print(y_star)
    write_file(fp,PART2_FN,y_star)
    print("End...")

# print(he)
# for k,v in he.items():
#     if k[0] == 'My':
#         print(k,v)

Part 2 running...


Doing EN/...
reading train file...

modifying to #UNK#...

building count(y->x) & count(y)...

calculating emission probability...

reading tests file...

doing simple_sentiment_analysis...

writing file...

End...
('My', 'O') 0.0014437752660671562
